In [2]:
#Pakages
using DifferentialEquations
using LaTeXStrings
using Plots; pyplot(); # here, pyplot is used as a back-end to the Plots API
using BenchmarkTools
using Polynomials
using LinearAlgebra

In [13]:
# utility parameters
LW1 = 2.5
LW2 = 1.5
LW3 = 1
LS1 = :solid
LS2 = :dot
LS3 = :dash
LS4 = :dashdot
LA1 = 1
LA2 = 0.7
LA3 = 0.4
# File system paths
#
# Figures are stored in this path.
Figpath = "C:\\Users\\kolds\\Dropbox\\Master's_Thesis\\MSc_Report_Lyx\\fig\\Simulation_Results\\Jupyter_Notebook_1\\" 
;

In [14]:
#  Utility functions
# -- convert vector of samples to vector of timeseries
equal_lengths(v) = all(x->length(x)==length(first(v)), v)
#
function vec2vec(v)
    equal_lengths(v) || error("The element vectors must have equal lengths.")
    return [getindex.(v,i) for i in firstindex(v[1]):lastindex(v[1])]
end

;

In [1]:
# Parameters
const chpa = 1.15	# Specific heat capacity of air, kJ.kg-1.K-1
const chpw = 4.2	# Specific heat capacity of water, kJ.kg-1.K-1

const hAax = 55.6	# Heat transfer, air side heat exchanger, kW/K
const hAwx = 222	# Heat transfer, water side heat exchanger, kW/K
#
# Heat exchanger length
# const Lx = 1.0 # x = 0 to 1 , Simpler, gives a relative position and can be omitted from the heat exchanger model.
# Because it has a value of 1, it was omitted from the model.;


In [4]:
# BL heat capacities
function cp_a(x)
    cp_a0 = 1.007
    y = x+273.15
    y0 = 300
    return cp_a0*(299.94 + 1.008*(y-y0)+1.2e-4*(y-y0)^2+5.9e-7*(y-y0)^3+2.56e-9*(y-y0)^4)/y
end
#
function cp_w(x)
    cp_w0 = 4.179
    y = x+273.15
    y0 = 300
    return cp_w0*(0.9998-6.6e-5*(y-y0) + 6.39e-6*(y-y0)^2-1.145e-7*(y-y0)^3+8.9e-10*(y-y0)^4)
end
;

cp_w (generic function with 1 method)

In [2]:
Twc = 3.8	# Cold water temperature, C
Tah = 39.12	# Hot air temperature, C
mdw = 53.9	# Heat exchanger water mass flow rate, kg/s
mda = 49.2	# Heat exchanger air mass flow rate, kg/s
const UAx = 1 / (1 / hAax + 1 / hAwx)	# Overall Heat transfer, heat exchanger, kW/K;

In [3]:
#analytic solution
function hex_a(inputs,par,x) # If x is an Array/tuple, the function can generate the temperature profile across x.
    Twc, Tah, mdw, mda, UAx = inputs
    chpw, chpa = par
    # Stanton numbers for air and water.
    NSta = UAx/chpa/mda	# Stanton number for air, -
    NStw = UAx/chpw/mdw	# Stanton number for water, -
    NStd = NStw - NSta	# Difference in Stanton numbers, -
    #
    Tac = (NStd*Tah + NSta*(1-exp(-NStd))*Twc)/(NStw-NSta*exp(-NStd))    
    Tw(x) = ((NStw*exp(-NStd*x)-NSta)*Twc+(NStw - NStw*exp(-NStd*x))*Tac)/NStd
    Ta(x) = ((NSta*exp(-NStd*x)-NSta)*Twc+(NStw - NSta*exp(-NStd*x))*Tac)/NStd
    return [Tw(x), Ta(x)]
end
;

In [12]:
# Parameters are passed to functions in an Array to avoid problems associated with global variables.
par_hex_a = [ chpw chpa]
par_hex_n_Cp_T_dep = [ cp_w cp_a] # Here, cp_w and cp_a are functions.
par_hex_n_Cp_T_indep = [ chpw chpa] # Here, chpw and chpa are constants.
#
# Inputs are passed to functions in an Array to avoid problems associated with global variables.
u_hex = [Twc Tah mdw mda UAx]
#
# Note that both the inputs and the parameters should be passed as the same type (Array or tuple) to reduce memory allocation for the numeric solver.
#
# The spatial span for the problem.
xspan_a = collect(0:0.01:1) # step size for the analytic model is specified here.
xspan_n = (0.0, 1.0) # step size for the Numeric model depends on the method used in the solver. Shooting method

UndefVarError: UndefVarError: cp_w not defined

In [6]:
sol_analytic = map(x -> hex_a(u_hex, par_hex_a, x), xspan_a) |> vec2vec;
plot(xspan_a,sol_analytic[1],lc=:blue,lw=LW1,ls=LS1,la=LA1,label="WaterTemperature");
plot!(xspan_a,sol_analytic[2],lc=:red,lw=LW1,ls=LS1,la=LA1,label="Air Temperature");

UndefVarError: UndefVarError: xspan_a not defined

In [4]:
#Numeric solution(Temperature dependent)
function hex_n_Cp_T_dep(inputs,par,x) 
    # The function for the ODE.
    function hex_i!(dy,y,par,x)
        # Interior of heat exchanger with temperature dependent heat capacity
        mdw, mda = par[3],par[4]
        UAx, cp_w, cp_a = par[5],par[6],par[7]
        #
        Tw,Ta = y
        dy[1] = -UAx/(mdw*cp_w(Tw))*(Tw - Ta)
        dy[2] = -UAx/(mda*cp_a(Ta))*(Tw - Ta)
    end
    #
    # Boundaries of heat exchanger with temperature dependent heat capacity
    function hex_b!(residual, y, par, x)
        Twc,Tah = par[1],par[2]
        residual[1] = y[1][1]-Twc	# y[1] is the beginning of the spatial span
        residual[2] = y[end][2]-Tah	# y[end] is the ending of the spatial span
    end
    #
    u0 = [0.0, 0.0] # The initial condition.
    prob_hex = BVProblem(hex_i!, hex_b!, u0, x, [inputs  par])
    sol_hex = solve(prob_hex, Shooting(Vern7()), dtmax=0.01) # Three solvers are available. Shooting is the fastest.
    return sol_hex # dtmax: Maximum dt for adaptive timestepping.
end
;


In [7]:
sol_numeric_Cp_T_dep = hex_n_Cp_T_dep(u_hex, par_hex_n_Cp_T_dep, xspan_n);


UndefVarError: UndefVarError: u_hex not defined

In [8]:
plot(sol_numeric_Cp_T_dep,vars=(0,1),lc=:blue,lw=LW1,ls=LS1,la=LA1,label="Water Temperature");
plot!(sol_numeric_Cp_T_dep,vars=(0,2),lc=:red,lw=LW1,ls=LS1,la=LA1,label="Air Temperature");

UndefVarError: UndefVarError: LW1 not defined

In [5]:
#Numeric solution(Temperature independent)
function hex_n_Cp_T_indep(inputs,par,x)
    # The function for the ODE.
    function hex_i!(dy,y,par,x)
        # Interior of heat exchanger with temperature independent heat capacity
        mdw, mda = par[3],par[4]
        UAx, cp_w, cp_a = par[5],par[6],par[7]
        #
        Tw,Ta = y
        dy[1] = -UAx/(mdw*cp_w)*(Tw - Ta)
        dy[2] = -UAx/(mda*cp_a)*(Tw - Ta)
    end
    #
    # Boundaries of heat exchanger with temperature independent heat capacity
    function hex_b!(residual, y, par, x)
        Twc,Tah = par[1],par[2]
        residual[1] = y[1][1]-Twc	# y[1] is the beginning of the spatial span
        residual[2] = y[end][2]-Tah	# y[end] is the ending of the spatial span
    end
    #
    u0 = [0.0, 0.0] # The initial condition.
    prob_hex = BVProblem(hex_i!, hex_b!, u0, x, [inputs  par])
    sol_hex = solve(prob_hex, Shooting(Vern7()), dtmax=0.01) # Three solvers are available. Shooting is the fastest.
    return sol_hex # dtmax: Maximum dt for adaptive timestepping.
end
;

In [9]:
sol_numeric_Cp_T_dep = hex_n_Cp_T_indep(u_hex, par_hex_n_Cp_T_dep, xspan_n);

UndefVarError: UndefVarError: u_hex not defined

In [11]:
plot(sol_numeric_Cp_T_indep,vars=(0,1),lc=:blue,lw=LW1,ls=LS1,la=LA1,label="Water Temperature");
plot!(sol_numeric_Cp_T_indep,vars=(0,2),lc=:red,lw=LW1,ls=LS1,la=LA1,label="Air Temperature");

UndefVarError: UndefVarError: LW1 not defined

In [10]:
#time calculation
@benchmark sol_analytic = map(x -> hex_a(u_hex, par_hex_a, x), xspan_a) |> vec2vec
@benchmark sol_numeric_Cp_T_dep = hex_n_Cp_T_dep(u_hex, par_hex_n_Cp_T_dep, xspan_n)
@benchmark sol_numeric_Cp_T_dep = hex_n_Cp_T_indep(u_hex, par_hex_n_Cp_T_dep, xspan_n)

LoadError: UndefVarError: @benchmark not defined